<a href="https://colab.research.google.com/github/madshjorth-andersen/Udrykning-og-beredskab/blob/main/v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Udrykning og beredskab

In [ ]:
import random
import pandas as pd

antal_biler = 20
uden_alarm = antal_biler // 2
med_alarm = antal_biler // 2

uden_alarm_reaktionstid = [random.randint(5, 20) for _ in range(uden_alarm)]
med_alarm_reaktionstid = [random.randint(15, 30) for _ in range(med_alarm)]

def forsinkelse_chance(reaktionstid, bilnummer):
    if reaktionstid > 25:
        resultat = "Forsinker ikke udrykningskøretøjet"
        chance = 0  # 0% chance for forsinkelse
    elif reaktionstid < 10:
        resultat = "Forsinker køretøjet"
        chance = 100  # 100% chance for forsinkelse
    else:
        resultat = "Chance for forsinkelse"
        chance = int((25 - reaktionstid) / 15 * 100)

    print(f"Bil {bilnummer}: reaktionstid {reaktionstid} - {resultat} ({chance}% chance for forsinkelse)")
    return chance

# Lister til at gemme forsinkelsesdata
forsinkelser_uden_alarm = []
for i, tid in enumerate(uden_alarm_reaktionstid):
    forsinkelser_uden_alarm.append(forsinkelse_chance(tid, i + 1))

forsinkelser_med_alarm = []
for i, tid in enumerate(med_alarm_reaktionstid):
    forsinkelser_med_alarm.append(forsinkelse_chance(tid, i + uden_alarm + 1))

print("Forsinkelser uden alarm:", forsinkelser_uden_alarm)
print("Forsinkelser med alarm:", forsinkelser_med_alarm)

Bil 1: reaktionstid 18 - Chance for forsinkelse (46% chance for forsinkelse)
Bil 2: reaktionstid 20 - Chance for forsinkelse (33% chance for forsinkelse)
Bil 3: reaktionstid 10 - Chance for forsinkelse (100% chance for forsinkelse)
Bil 4: reaktionstid 20 - Chance for forsinkelse (33% chance for forsinkelse)
Bil 5: reaktionstid 14 - Chance for forsinkelse (73% chance for forsinkelse)
Bil 6: reaktionstid 12 - Chance for forsinkelse (86% chance for forsinkelse)
Bil 7: reaktionstid 15 - Chance for forsinkelse (66% chance for forsinkelse)
Bil 8: reaktionstid 8 - Forsinker køretøjet (100% chance for forsinkelse)
Bil 9: reaktionstid 15 - Chance for forsinkelse (66% chance for forsinkelse)
Bil 10: reaktionstid 20 - Chance for forsinkelse (33% chance for forsinkelse)
Bil 11: reaktionstid 15 - Chance for forsinkelse (66% chance for forsinkelse)
Bil 12: reaktionstid 20 - Chance for forsinkelse (33% chance for forsinkelse)
Bil 13: reaktionstid 30 - Forsinker ikke udrykningskøretøjet (0% chance for

#Koden til kort-visualisering

In [ ]:
import json
import folium
import time
from shapely.geometry import LineString, Point
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output

# Constants
TIME_STEP = 1  # seconds per update
EPSG_UTM = 25832  # UTM Zone for Denmark
EPSG_WGS = 4326   # WGS84

# Function to load GeoJSON LineString from a local file
def load_road_from_geojson(file_path):
    """Loads a LineString from a locally stored GeoJSON file."""
    with open(file_path, 'r') as file:
        geojson_data = json.load(file)

    # Extract coordinates assuming a single LineString
    coordinates = geojson_data['features'][0]['geometry']['coordinates']
    return LineString(coordinates)

# Function to transform coordinates between WGS84 and UTM
def transform_to_utm(line, from_epsg=EPSG_WGS, to_epsg=EPSG_UTM):
    """Transforms a LineString from WGS84 to UTM coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, line)

def transform_to_wgs(point, from_epsg=EPSG_UTM, to_epsg=EPSG_WGS):
    """Transforms a Point from UTM to WGS84 coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, point)

class Car:
    """A simple car simulation moving along a road."""

    def __init__(self, road: LineString, speed_kmh: float):
        """Initializes the car simulation."""
        self.road = road
        self.speed_mps = self.kmh_to_mps(speed_kmh)
        self.position_m = 0  # Start at the beginning of the road
        self.road_length = road.length

    def kmh_to_mps(self, speed_kmh):
        """Converts speed from km/h to m/s."""
        return speed_kmh * 1000 / 3600

    def advance(self, time_step):
        """Advances the car's position based on speed and time step."""
        self.position_m += self.speed_mps * time_step
        self.position_m = min(self.position_m, self.road_length)  # Prevent overshooting

    def get_position(self):
        """Returns the current position of the car as a Shapely Point."""
        return self.road.interpolate(self.position_m)

    def has_reached_end(self):
        """Checks if the car has reached the end of the road."""
        return self.position_m >= self.road_length

# Main simulation function
def simulate_car_movement(geojson_path, speed_kmh):
    """Simulates a car moving along a locally stored road."""

    # Load road and convert to UTM
    road_wgs = load_road_from_geojson(geojson_path)
    road_utm = transform_to_utm(road_wgs)

    # Initialize car
    car = Car(road_utm, speed_kmh)

    # Setup Folium map centered at start point
    start_point_wgs = transform_to_wgs(road_utm.interpolate(0))
    m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15)

    # Add road to map
    folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

    # Simulation loop
    while not car.has_reached_end():
        car.advance(TIME_STEP)
        car_pos_wgs = transform_to_wgs(car.get_position())

        # Clear previous markers and add new car icon as a marker
        m = folium.Map(location=[car_pos_wgs.y, car_pos_wgs.x], zoom_start=15)
        folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

        folium.Marker(
            location=[car_pos_wgs.y, car_pos_wgs.x],
            icon=folium.Icon(color="red", icon="car", prefix="fa"),  # Using Font Awesome "car" icon
            popup=f"Speed: {speed_kmh} km/h"
        ).add_to(m)

        # Save map to HTML and reload output
        m.save("car_simulation.html")
        clear_output(wait=True)
        display(m)

        # Print progress
        print(f"Car position: {car.position_m:.2f} meters / {car.road_length:.2f} meters")

        time.sleep(0.5)  # Simulate real-time updates

    print("Car has reached the end of the road.")

# Run simulation with locally saved file
geojson_path = "sample_data/map.geojson"  # Update the path to your map.geojson file
simulate_car_movement(geojson_path, speed_kmh=120)


Car position: 4473.60 meters / 4473.60 meters
Car has reached the end of the road.


In [ ]:
import json
import folium
import time
import random
from shapely.geometry import LineString, Point
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output

# Constants
TIME_STEP = 1  # seconds per update
EPSG_UTM = 25832  # UTM Zone for Denmark
EPSG_WGS = 4326   # WGS84
ALARM_SPEED_INCREASE = 1.2  # 20% faster for cars with alarms
NO_ALARM_SPEED_DECREASE = 0.8  # 20% slower for cars without alarms
FAST_CAR_SPEED_KMH = 150  # Speed of the faster car in km/h

# Function to load GeoJSON LineString from a local file
def load_road_from_geojson(file_path):
    """Loads a LineString from a locally stored GeoJSON file."""
    with open(file_path, 'r') as file:
        geojson_data = json.load(file)

    # Extract coordinates assuming a single LineString
    coordinates = geojson_data['features'][0]['geometry']['coordinates']
    return LineString(coordinates)

# Function to transform coordinates between WGS84 and UTM
def transform_to_utm(line, from_epsg=EPSG_WGS, to_epsg=EPSG_UTM):
    """Transforms a LineString from WGS84 to UTM coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, line)

def transform_to_wgs(point, from_epsg=EPSG_UTM, to_epsg=EPSG_WGS):
    """Transforms a Point from UTM to WGS84 coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, point)

class Car:
    """A simple car simulation moving along a road."""

    def __init__(self, road: LineString, speed_kmh: float, has_alarm: bool, delay_chance: int):
        """Initializes the car simulation."""
        self.road = road
        self.speed_mps = self.kmh_to_mps(speed_kmh)
        self.position_m = 0  # Start at the beginning of the road
        self.road_length = road.length
        self.has_alarm = has_alarm
        self.delay_chance = delay_chance

        # Apply delay chance: If there's a delay, reduce the speed
        if self.has_alarm:
            self.speed_mps *= ALARM_SPEED_INCREASE  # Increase speed if car has an alarm
        else:
            # Delay effect for cars without alarms
            if random.randint(0, 100) < self.delay_chance:  # Delay happens based on chance
                self.speed_mps *= NO_ALARM_SPEED_DECREASE  # Reduce speed

    def kmh_to_mps(self, speed_kmh):
        """Converts speed from km/h to m/s."""
        return speed_kmh * 1000 / 3600

    def advance(self, time_step):
        """Advances the car's position based on speed and time step."""
        self.position_m += self.speed_mps * time_step
        self.position_m = min(self.position_m, self.road_length)  # Prevent overshooting

    def get_position(self):
        """Returns the current position of the car as a Shapely Point."""
        return self.road.interpolate(self.position_m)

    def has_reached_end(self):
        """Checks if the car has reached the end of the road."""
        return self.position_m >= self.road_length

# Main simulation function
def simulate_car_movement(geojson_path, speed_kmh=120):
    """Simulates multiple cars moving along a locally stored road."""

    # Load road and convert to UTM
    road_wgs = load_road_from_geojson(geojson_path)
    road_utm = transform_to_utm(road_wgs)

    # Create Car A (with alarm) and Car B (without alarm)
    car_a = Car(road_utm, speed_kmh, has_alarm=True, delay_chance=0)  # Car with alarm, no delay
    car_b = Car(road_utm, speed_kmh, has_alarm=False, delay_chance=random.randint(10, 50))  # Car without alarm, possible delay

    # Create Fast Car (Car C)
    fast_car = Car(road_utm, FAST_CAR_SPEED_KMH, has_alarm=False, delay_chance=0)  # Fast car, constant speed

    # Setup Folium map centered at start point
    start_point_wgs = transform_to_wgs(road_utm.interpolate(0))
    m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15)

    # Add road to map
    folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

    # Simulation loop
    while not (car_a.has_reached_end() and car_b.has_reached_end()):
        car_a.advance(TIME_STEP)
        car_b.advance(TIME_STEP)
        fast_car.advance(TIME_STEP)

        # Clear previous markers and add new car icons as markers
        m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15)
        folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

        # Add car markers
        for car, color in [(car_a, "green"), (car_b, "red"), (fast_car, "blue")]:
            car_pos_wgs = transform_to_wgs(car.get_position())
            folium.Marker(
                location=[car_pos_wgs.y, car_pos_wgs.x],
                icon=folium.Icon(color=color, icon="car", prefix="fa"),  # Car icon
                popup=f"Speed: {speed_kmh} km/h, Alarm: {'Yes' if car.has_alarm else 'No'}"
            ).add_to(m)

        # Save map to HTML and reload output
        m.save("car_simulation.html")
        clear_output(wait=True)
        display(m)

        # Print progress
        print(f"Car A position: {car_a.position_m:.2f} meters / {car_a.road_length:.2f} meters")
        print(f"Car B position: {car_b.position_m:.2f} meters / {car_b.road_length:.2f} meters")
        print(f"Fast Car position: {fast_car.position_m:.2f} meters / {fast_car.road_length:.2f} meters")

        time.sleep(0.5)  # Simulate real-time updates

    print("Both Car A and Car B have reached the end of the road.")

# Run simulation with locally saved file
geojson_path = "sample_data/map.geojson"  # Update the path to your map.geojson file
simulate_car_movement(geojson_path, speed_kmh=120)


Car A position: 1080.00 meters / 4473.60 meters
Car B position: 900.00 meters / 4473.60 meters
Fast Car position: 1125.00 meters / 4473.60 meters


KeyboardInterrupt: 

In [ ]:
import json
import folium
import time
import random
from shapely.geometry import LineString, Point
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output

# Constants
TIME_STEP = 1  # seconds per update
EPSG_UTM = 25832  # UTM Zone for Denmark
EPSG_WGS = 4326   # WGS84
ALARM_SPEED_INCREASE = 1.2  # 20% faster for cars with alarms
NO_ALARM_SPEED_DECREASE = 0.8  # 20% slower for cars without alarms
FAST_CAR_SPEED_KMH = 150  # Speed of the faster car in km/h

# Function to load GeoJSON LineString from a local file
def load_road_from_geojson(file_path):
    """Loads a LineString from a locally stored GeoJSON file."""
    with open(file_path, 'r') as file:
        geojson_data = json.load(file)

    # Extract coordinates assuming a single LineString
    coordinates = geojson_data['features'][0]['geometry']['coordinates']
    return LineString(coordinates)

# Function to transform coordinates between WGS84 and UTM
def transform_to_utm(line, from_epsg=EPSG_WGS, to_epsg=EPSG_UTM):
    """Transforms a LineString from WGS84 to UTM coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, line)

def transform_to_wgs(point, from_epsg=EPSG_UTM, to_epsg=EPSG_WGS):
    """Transforms a Point from UTM to WGS84 coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, point)

class Car:
    """A simple car simulation moving along a road."""

    def __init__(self, road: LineString, speed_kmh: float, has_alarm: bool, delay_chance: int):
        """Initializes the car simulation."""
        self.road = road
        self.speed_mps = self.kmh_to_mps(speed_kmh)
        self.position_m = 0  # Start at the beginning of the road
        self.road_length = road.length
        self.has_alarm = has_alarm
        self.delay_chance = delay_chance

        # Apply delay chance: If there's a delay, reduce the speed
        if self.has_alarm:
            self.speed_mps *= ALARM_SPEED_INCREASE  # Increase speed if car has an alarm
        else:
            # Delay effect for cars without alarms
            if random.randint(0, 100) < self.delay_chance:  # Delay happens based on chance
                self.speed_mps *= NO_ALARM_SPEED_DECREASE  # Reduce speed

    def kmh_to_mps(self, speed_kmh):
        """Converts speed from km/h to m/s."""
        return speed_kmh * 1000 / 3600

    def advance(self, time_step):
        """Advances the car's position based on speed and time step."""
        self.position_m += self.speed_mps * time_step
        self.position_m = min(self.position_m, self.road_length)  # Prevent overshooting

    def get_position(self):
        """Returns the current position of the car as a Shapely Point."""
        return self.road.interpolate(self.position_m)

    def has_reached_end(self):
        """Checks if the car has reached the end of the road."""
        return self.position_m >= self.road_length

# Main simulation function
def simulate_car_movement(geojson_path, speed_kmh=120):
    """Simulates multiple cars moving along a locally stored road."""

    # Load road and convert to UTM
    road_wgs = load_road_from_geojson(geojson_path)
    road_utm = transform_to_utm(road_wgs)

    # Create Car A (with alarm) and Car B (without alarm)
    car_a = Car(road_utm, speed_kmh, has_alarm=True, delay_chance=0)  # Car with alarm, no delay
    car_b = Car(road_utm, speed_kmh, has_alarm=False, delay_chance=random.randint(10, 50))  # Car without alarm, possible delay

    # Create Fast Car (Car C)
    fast_car = Car(road_utm, FAST_CAR_SPEED_KMH, has_alarm=False, delay_chance=0)  # Fast car, constant speed

    # Setup Folium map centered at start point
    start_point_wgs = transform_to_wgs(road_utm.interpolate(0))
    m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15)

    # Add road to map
    folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

    # Simulation loop
    while not (car_a.has_reached_end() and car_b.has_reached_end()):
        car_a.advance(TIME_STEP)
        car_b.advance(TIME_STEP)
        fast_car.advance(TIME_STEP)

        # Clear previous markers and add new car icons as markers
        m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15, control_scale=True)
        folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

        # Add car markers (using FontAwesome icons for car and police)
        for car, color, icon in [(car_a, "green", "car"), (car_b, "red", "car"), (fast_car, "blue", "police")]:
            car_pos_wgs = transform_to_wgs(car.get_position())
            folium.Marker(
                location=[car_pos_wgs.y, car_pos_wgs.x],
                icon=folium.Icon(color=color, icon=icon, prefix="fa"),  # Use FontAwesome car and police icons
                popup=f"Speed: {speed_kmh} km/h, Alarm: {'Yes' if car.has_alarm else 'No'}"
            ).add_to(m)

        # Dynamically adjust the zoom level and map center based on car positions
        car_positions = [transform_to_wgs(car.get_position()) for car in [car_a, car_b, fast_car]]
        latitudes = [pos.y for pos in car_positions]
        longitudes = [pos.x for pos in car_positions]
        avg_lat = sum(latitudes) / len(latitudes)
        avg_lon = sum(longitudes) / len(longitudes)
        max_distance = max([car_a.position_m, car_b.position_m, fast_car.position_m])
        zoom_level = 15 if max_distance < road_utm.length * 0.5 else 14  # Adjust zoom based on distance

        m.location = [avg_lat, avg_lon]  # Center map at average position of all cars
        m.zoom_start = zoom_level  # Adjust zoom

        # Save map to HTML and reload output
        m.save("car_simulation.html")
        clear_output(wait=True)
        display(m)

        # Print progress
        print(f"Car A position: {car_a.position_m:.2f} meters / {car_a.road_length:.2f} meters")
        print(f"Car B position: {car_b.position_m:.2f} meters / {car_b.road_length:.2f} meters")
        print(f"Fast Car position: {fast_car.position_m:.2f} meters / {fast_car.road_length:.2f} meters")

        time.sleep(0.5)  # Simulate real-time updates

    print("Both Car A and Car B have reached the end of the road.")

# Run simulation with locally saved file
geojson_path = "sample_data/map.geojson"  # Update the path to your map.geojson file
simulate_car_movement(geojson_path, speed_kmh=120)


Car A position: 1720.00 meters / 4473.60 meters
Car B position: 1433.33 meters / 4473.60 meters
Fast Car position: 1791.67 meters / 4473.60 meters


KeyboardInterrupt: 

In [ ]:
import json
import folium
import time
import random
from shapely.geometry import LineString, Point
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output

# Constants
TIME_STEP = 1  # seconds per update
EPSG_UTM = 25832  # UTM Zone for Denmark
EPSG_WGS = 4326   # WGS84
ALARM_SPEED_INCREASE = 1.2  # 20% faster for cars with alarms
NO_ALARM_SPEED_DECREASE = 0.8  # 20% slower for cars without alarms
FAST_CAR_SPEED_KMH = 150  # Speed of the faster car in km/h

# Function to load GeoJSON LineString from a local file
def load_road_from_geojson(file_path):
    """Loads a LineString from a locally stored GeoJSON file."""
    with open(file_path, 'r') as file:
        geojson_data = json.load(file)

    # Extract coordinates assuming a single LineString
    coordinates = geojson_data['features'][0]['geometry']['coordinates']
    return LineString(coordinates)

# Function to transform coordinates between WGS84 and UTM
def transform_to_utm(line, from_epsg=EPSG_WGS, to_epsg=EPSG_UTM):
    """Transforms a LineString from WGS84 to UTM coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, line)

def transform_to_wgs(point, from_epsg=EPSG_UTM, to_epsg=EPSG_WGS):
    """Transforms a Point from UTM to WGS84 coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, point)

class Car:
    """A simple car simulation moving along a road."""

    def __init__(self, road: LineString, speed_kmh: float, has_alarm: bool, delay_chance: int, start_delay: float = 0):
        """Initializes the car simulation."""
        self.road = road
        self.speed_mps = self.kmh_to_mps(speed_kmh)
        self.position_m = 0  # Start at the beginning of the road
        self.road_length = road.length
        self.has_alarm = has_alarm
        self.delay_chance = delay_chance
        self.start_delay = start_delay  # Delay for the fastest car to start

        # Apply delay chance: If there's a delay, reduce the speed
        if self.has_alarm:
            self.speed_mps *= ALARM_SPEED_INCREASE  # Increase speed if car has an alarm
        else:
            # Delay effect for cars without alarms
            if random.randint(0, 100) < self.delay_chance:  # Delay happens based on chance
                self.speed_mps *= NO_ALARM_SPEED_DECREASE  # Reduce speed

    def kmh_to_mps(self, speed_kmh):
        """Converts speed from km/h to m/s."""
        return speed_kmh * 1000 / 3600

    def advance(self, time_step):
        """Advances the car's position based on speed and time step."""
        self.position_m += self.speed_mps * time_step
        self.position_m = min(self.position_m, self.road_length)  # Prevent overshooting

    def get_position(self):
        """Returns the current position of the car as a Shapely Point."""
        return self.road.interpolate(self.position_m)

    def has_reached_end(self):
        """Checks if the car has reached the end of the road."""
        return self.position_m >= self.road_length

# Main simulation function
def simulate_car_movement(geojson_path, speed_kmh=120):
    """Simulates multiple cars moving along a locally stored road."""

    # Load road and convert to UTM
    road_wgs = load_road_from_geojson(geojson_path)
    road_utm = transform_to_utm(road_wgs)

    # Create Car A (with alarm) and Car B (without alarm)
    car_a = Car(road_utm, speed_kmh, has_alarm=True, delay_chance=0)  # Car with alarm, no delay
    car_b = Car(road_utm, speed_kmh, has_alarm=False, delay_chance=random.randint(10, 50))  # Car without alarm, possible delay

    # Create Fast Car (Car C) with delayed start
    fast_car = Car(road_utm, FAST_CAR_SPEED_KMH, has_alarm=False, delay_chance=0, start_delay=5)  # Fast car, starts later

    # Setup Folium map centered at start point
    start_point_wgs = transform_to_wgs(road_utm.interpolate(0))
    m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15)

    # Add road to map
    folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

    # Simulation loop
    time_elapsed = 0  # Track the overall time to simulate start delay for the fastest car
    while not (car_a.has_reached_end() and car_b.has_reached_end()):
        # Introduce delay for the fast car to start
        if time_elapsed >= fast_car.start_delay:
            fast_car.advance(TIME_STEP)

        car_a.advance(TIME_STEP)
        car_b.advance(TIME_STEP)

        # Clear previous markers and add new car icons as markers
        m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15, control_scale=True)
        folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

        # Add car markers (using FontAwesome icons for car and police)
        for car, color, icon in [(car_a, "green", "car"), (car_b, "red", "car"), (fast_car, "blue", "police")]:
            car_pos_wgs = transform_to_wgs(car.get_position())
            folium.Marker(
                location=[car_pos_wgs.y, car_pos_wgs.x],
                icon=folium.Icon(color=color, icon=icon, prefix="fa"),  # Use FontAwesome car and police icons
                popup=f"Speed: {speed_kmh} km/h, Alarm: {'Yes' if car.has_alarm else 'No'}"
            ).add_to(m)

        # Dynamically adjust the zoom level and map center based on car positions
        car_positions = [transform_to_wgs(car.get_position()) for car in [car_a, car_b, fast_car]]
        latitudes = [pos.y for pos in car_positions]
        longitudes = [pos.x for pos in car_positions]
        avg_lat = sum(latitudes) / len(latitudes)
        avg_lon = sum(longitudes) / len(longitudes)
        max_distance = max([car_a.position_m, car_b.position_m, fast_car.position_m])
        zoom_level = 15 if max_distance < road_utm.length * 0.5 else 14  # Adjust zoom based on distance

        m.location = [avg_lat, avg_lon]  # Center map at average position of all cars
        m.zoom_start = zoom_level  # Adjust zoom

        # Save map to HTML and reload output
        m.save("car_simulation.html")
        clear_output(wait=True)
        display(m)

        # Print progress
        print(f"Car A position: {car_a.position_m:.2f} meters / {car_a.road_length:.2f} meters")
        print(f"Car B position: {car_b.position_m:.2f} meters / {car_b.road_length:.2f} meters")
        print(f"Fast Car position: {fast_car.position_m:.2f} meters / {fast_car.road_length:.2f} meters")

        time_elapsed += TIME_STEP  # Increment the total time
        time.sleep(0.5)  # Simulate real-time updates

    print("All cars have reached the end of the road.")

# Run simulation with locally saved file
geojson_path = "sample_data/map.geojson"  # Update the path to your map.geojson file
simulate_car_movement(geojson_path, speed_kmh=120)


Car A position: 4473.60 meters / 4473.60 meters
Car B position: 4473.60 meters / 4473.60 meters
Fast Car position: 4473.60 meters / 4473.60 meters
All cars have reached the end of the road.


In [ ]:
import json
import folium
import time
import random
from shapely.geometry import LineString, Point
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output

# Constants
TIME_STEP = 1  # seconds per update
EPSG_UTM = 25832  # UTM Zone for Denmark
EPSG_WGS = 4326   # WGS84
ALARM_SPEED_INCREASE = 1.2  # 20% faster for cars with alarms
NO_ALARM_SPEED_DECREASE = 0.8  # 20% slower for cars without alarms
FAST_CAR_SPEED_KMH = 180  # Speed of the police car in km/h (faster than the others)

# Function to load GeoJSON LineString from a local file
def load_road_from_geojson(file_path):
    """Loads a LineString from a locally stored GeoJSON file."""
    with open(file_path, 'r') as file:
        geojson_data = json.load(file)

    # Extract coordinates assuming a single LineString
    coordinates = geojson_data['features'][0]['geometry']['coordinates']
    return LineString(coordinates)

# Function to transform coordinates between WGS84 and UTM
def transform_to_utm(line, from_epsg=EPSG_WGS, to_epsg=EPSG_UTM):
    """Transforms a LineString from WGS84 to UTM coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, line)

def transform_to_wgs(point, from_epsg=EPSG_UTM, to_epsg=EPSG_WGS):
    """Transforms a Point from UTM to WGS84 coordinates."""
    transformer = pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform
    return transform(transformer, point)

class Car:
    """A simple car simulation moving along a road."""

    def __init__(self, road: LineString, speed_kmh: float, has_alarm: bool, delay_chance: int, start_delay: float = 0):
        """Initializes the car simulation."""
        self.road = road
        self.speed_mps = self.kmh_to_mps(speed_kmh)
        self.position_m = 0  # Start at the beginning of the road
        self.road_length = road.length
        self.has_alarm = has_alarm
        self.delay_chance = delay_chance
        self.start_delay = start_delay  # Delay for the fastest car to start

        # Apply delay chance: If there's a delay, reduce the speed
        if self.has_alarm:
            self.speed_mps *= ALARM_SPEED_INCREASE  # Increase speed if car has an alarm
        else:
            # Delay effect for cars without alarms
            if random.randint(0, 100) < self.delay_chance:  # Delay happens based on chance
                self.speed_mps *= NO_ALARM_SPEED_DECREASE  # Reduce speed

    def kmh_to_mps(self, speed_kmh):
        """Converts speed from km/h to m/s."""
        return speed_kmh * 1000 / 3600

    def advance(self, time_step):
        """Advances the car's position based on speed and time step."""
        self.position_m += self.speed_mps * time_step
        self.position_m = min(self.position_m, self.road_length)  # Prevent overshooting

    def get_position(self):
        """Returns the current position of the car as a Shapely Point."""
        return self.road.interpolate(self.position_m)

    def has_reached_end(self):
        """Checks if the car has reached the end of the road."""
        return self.position_m >= self.road_length

# Main simulation function
def simulate_car_movement(geojson_path, speed_kmh=120):
    """Simulates multiple cars moving along a locally stored road."""

    # Load road and convert to UTM
    road_wgs = load_road_from_geojson(geojson_path)
    road_utm = transform_to_utm(road_wgs)

    # Create Car A (with alarm) and Car B (without alarm)
    car_a = Car(road_utm, speed_kmh, has_alarm=True, delay_chance=0)  # Car with alarm, no delay
    car_b = Car(road_utm, speed_kmh, has_alarm=False, delay_chance=random.randint(10, 50))  # Car without alarm, possible delay

    # Create Fast Car (Car C) with delayed start (faster)
    fast_car = Car(road_utm, FAST_CAR_SPEED_KMH, has_alarm=False, delay_chance=0, start_delay=5)  # Fast car, starts later

    # Setup Folium map centered at start point
    start_point_wgs = transform_to_wgs(road_utm.interpolate(0))
    m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15)

    # Add road to map
    folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

    # Custom car icons (file paths for local images)
    red_car_icon = folium.CustomIcon(icon_image='sample_data/redcar.png', icon_size=(40, 40))  # Red car image
    blue_car_icon = folium.CustomIcon(icon_image='sample_data/bluecar.png', icon_size=(40, 40))  # Blue car image
    police_car_icon = folium.CustomIcon(icon_image='sample_data/policecar.png', icon_size=(50, 50))  # Police car image

    # Simulation loop
    time_elapsed = 0  # Track the overall time to simulate start delay for the fastest car
    while not (car_a.has_reached_end() and car_b.has_reached_end()):
        # Introduce delay for the fast car to start
        if time_elapsed >= fast_car.start_delay:
            fast_car.advance(TIME_STEP)

        car_a.advance(TIME_STEP)
        car_b.advance(TIME_STEP)

        # Clear previous markers and add new car icons as markers
        m = folium.Map(location=[start_point_wgs.y, start_point_wgs.x], zoom_start=15, control_scale=True)
        folium.PolyLine([(pt[1], pt[0]) for pt in road_wgs.coords], color="blue", weight=3).add_to(m)

        # Add car markers (using custom images for each car)
        for car, icon in [(car_a, red_car_icon), (car_b, blue_car_icon), (fast_car, police_car_icon)]:
            car_pos_wgs = transform_to_wgs(car.get_position())
            folium.Marker(
                location=[car_pos_wgs.y, car_pos_wgs.x],
                icon=icon,
                popup=f"Speed: {speed_kmh} km/h, Alarm: {'Yes' if car.has_alarm else 'No'}"
            ).add_to(m)

        # Dynamically adjust the zoom level and map center based on car positions
        car_positions = [transform_to_wgs(car.get_position()) for car in [car_a, car_b, fast_car]]
        latitudes = [pos.y for pos in car_positions]
        longitudes = [pos.x for pos in car_positions]
        avg_lat = sum(latitudes) / len(latitudes)
        avg_lon = sum(longitudes) / len(longitudes)
        max_distance = max([car_a.position_m, car_b.position_m, fast_car.position_m])
        zoom_level = 15 if max_distance < road_utm.length * 0.5 else 14  # Adjust zoom based on distance

        m.location = [avg_lat, avg_lon]  # Center map at average position of all cars
        m.zoom_start = zoom_level  # Adjust zoom

        # Save map to HTML and reload output
        m.save("car_simulation.html")
        clear_output(wait=True)
        display(m)

        # Print progress
        print(f"Car A position: {car_a.position_m:.2f} meters / {car_a.road_length:.2f} meters")
        print(f"Car B position: {car_b.position_m:.2f} meters / {car_b.road_length:.2f} meters")
        print(f"Fast Car position: {fast_car.position_m:.2f} meters / {fast_car.road_length:.2f} meters")

        time_elapsed += TIME_STEP  # Increment the total time
        time.sleep(0.5)  # Simulate real-time updates

    print("All cars have reached the end of the road.")

# Run simulation with locally saved file
geojson_path = "sample_data/map.geojson"  # Update the path to your map.geojson file
simulate_car_movement(geojson_path, speed_kmh=120)


FileNotFoundError: [Errno 2] No such file or directory: 'sample_data/map.geojson'

In [8]:
import json
import folium
import time
import random
from shapely.geometry import LineString
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output

# Constants
TIME_STEP = 0.5  # Reduced time per update for faster simulation
ALARM_SPEED_MULTIPLIER = 1.2
NO_ALARM_SPEED_MULTIPLIER = 0.8
FAST_CAR_SPEED_KMH = 180
FAST_CAR_DELAY = 3  # Fast car starts slightly sooner

# Load road from GeoJSON
def load_road(file_path):
    with open(file_path, 'r') as file:
        return LineString(json.load(file)['features'][0]['geometry']['coordinates'])

# Transform coordinates
def transform_coords(geom, from_epsg=4326, to_epsg=25832):
    return transform(pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform, geom)

# Car class
class Car:
    def __init__(self, road, speed_kmh, has_alarm=False, delay_chance=0, start_delay=0):
        self.road = transform_coords(road)
        self.speed_mps = (speed_kmh * 1000 / 3600) * (ALARM_SPEED_MULTIPLIER if has_alarm else NO_ALARM_SPEED_MULTIPLIER if random.randint(0, 100) < delay_chance else 1)
        self.position_m = 0
        self.start_delay = start_delay

    def advance(self):
        self.position_m = min(self.position_m + self.speed_mps * TIME_STEP, self.road.length)

    def get_position(self):
        return transform_coords(self.road.interpolate(self.position_m), from_epsg=25832, to_epsg=4326)

# Simulation
def run_simulation(geojson_path, speed_kmh=120):
    road = load_road(geojson_path)

    car_a = Car(road, speed_kmh, has_alarm=True)
    car_b = Car(road, speed_kmh, delay_chance=random.randint(10, 50))
    fast_car = Car(road, FAST_CAR_SPEED_KMH, start_delay=FAST_CAR_DELAY)

    icons = {
        car_a: folium.CustomIcon("sample_data/redcar.png", icon_size=(40, 40)),
        car_b: folium.CustomIcon("sample_data/bluecar.png", icon_size=(40, 40)),
        fast_car: folium.CustomIcon("sample_data/policecar.png", icon_size=(50, 50)),
    }

    time_elapsed = 0
    while car_a.position_m < car_a.road.length or car_b.position_m < car_b.road.length:
        if time_elapsed >= fast_car.start_delay:
            fast_car.advance()
        car_a.advance()
        car_b.advance()

        map_center = car_a.get_position()
        m = folium.Map(location=[map_center.y, map_center.x], zoom_start=15, control_scale=True)
        folium.PolyLine([(pt[1], pt[0]) for pt in road.coords], color="blue", weight=3).add_to(m)

        for car in [car_a, car_b, fast_car]:
            folium.Marker(
                location=[car.get_position().y, car.get_position().x], icon=icons[car]
            ).add_to(m)

        m.save("car_simulation.html")
        clear_output(wait=True)
        display(m)

        time_elapsed += TIME_STEP
        time.sleep(0.25)  # Faster updates

    print("Simulation complete.")

# Run the simulation
run_simulation("sample_data/map.geojson", speed_kmh=120)


KeyboardInterrupt: 

In [9]:
import json
import folium
import time
import random
from shapely.geometry import LineString
from shapely.ops import transform
import pyproj
from IPython.display import display, clear_output

# Constants
TIME_STEP = 0.2  # More frequent updates
ALARM_SPEED_MULTIPLIER = 1.3  # Increase speed boost for alarm cars
NO_ALARM_SPEED_MULTIPLIER = 0.7  # Further slow down cars without alarms
FAST_CAR_SPEED_KMH = 200  # Police car is even faster
FAST_CAR_DELAY = 2  # Starts even sooner

# Load road from GeoJSON
def load_road(file_path):
    with open(file_path, 'r') as file:
        return LineString(json.load(file)['features'][0]['geometry']['coordinates'])

# Coordinate transformation
def transform_coords(geom, from_epsg=4326, to_epsg=25832):
    return transform(pyproj.Transformer.from_crs(from_epsg, to_epsg, always_xy=True).transform, geom)

# Car class
class Car:
    def __init__(self, road, speed_kmh, has_alarm=False, delay_chance=0, start_delay=0):
        self.road = transform_coords(road)
        self.speed_mps = (speed_kmh * 1000 / 3600) * (ALARM_SPEED_MULTIPLIER if has_alarm else NO_ALARM_SPEED_MULTIPLIER if random.randint(0, 100) < delay_chance else 1)
        self.position_m = 0
        self.start_delay = start_delay

    def advance(self):
        self.position_m = min(self.position_m + self.speed_mps * TIME_STEP, self.road.length)

    def get_position(self):
        return transform_coords(self.road.interpolate(self.position_m), from_epsg=25832, to_epsg=4326)

# Run simulation
def run_simulation(geojson_path, speed_kmh=120):
    road = load_road(geojson_path)

    car_a = Car(road, speed_kmh, has_alarm=True)
    car_b = Car(road, speed_kmh, delay_chance=random.randint(10, 50))
    fast_car = Car(road, FAST_CAR_SPEED_KMH, start_delay=FAST_CAR_DELAY)

    icons = {
        car_a: folium.CustomIcon("sample_data/redcar.png", icon_size=(40, 40)),
        car_b: folium.CustomIcon("sample_data/bluecar.png", icon_size=(40, 40)),
        fast_car: folium.CustomIcon("sample_data/policecar.png", icon_size=(50, 50)),
    }

    time_elapsed = 0
    while car_a.position_m < car_a.road.length or car_b.position_m < car_b.road.length:
        if time_elapsed >= fast_car.start_delay:
            fast_car.advance()
        car_a.advance()
        car_b.advance()

        map_center = car_a.get_position()
        m = folium.Map(location=[map_center.y, map_center.x], zoom_start=15, control_scale=True)
        folium.PolyLine([(pt[1], pt[0]) for pt in road.coords], color="blue", weight=3).add_to(m)

        for car in [car_a, car_b, fast_car]:
            folium.Marker(
                location=[car.get_position().y, car.get_position().x], icon=icons[car]
            ).add_to(m)

        m.save("car_simulation.html")
        clear_output(wait=True)
        display(m)

        time_elapsed += TIME_STEP
        time.sleep(0.1)  # Even faster updates

    print("Simulation complete.")

# Run the simulation
run_simulation("sample_data/map.geojson", speed_kmh=120)


KeyboardInterrupt: 